## App Layout & Callbacks

In the previous notebook (`00_project_startup.ipynb`), we prepared almost all the backend logic:
* We loaded the data and organized it by tab
* We computed KPI values for our stat cards
* We set up dictionaries to map tabs to builder functions
* We defined reusable style and layout helpers

In this notebook, we now bring everything **together in the actual Dash app**.  
We'll walk through the full structure of app.py, focusing on two key elements:

1. **The Layout** – How we build the visual page using Dash + Bootstrap components (e.g., `dbc.Container`, `dcc.Tabs`, `dbc.Row`, and `dbc.Col`)
2. **The Callback Logic** – How tab selections dynamically control what graphs are shown, and how we route those through helper functions and builders in a clean, scalable way.

We'll cover:
* How the layout is structured using Dash and Bootstrap (logo, KPI cards, tab bars, and figures)
* How a single `@app.callback` connects the selected tabs to the correct data and visualisations
* How to add new tabs or views by updating mappings without changing core logic

We’ll go through the layout row by row, and then explain the callback that powers the dynamic behaviour.


---

We'll start by initializing the dash app as follows:

In [5]:
from pathlib import Path

import dash_bootstrap_components as dbc
import pandas as pd
from dash import Dash, dcc, html, Input, Output  # State not currently needed

from src.const import get_constants
from src import dash1, dash2, dash3, dash4

# ──────────────────────────────────────────────────────────────────────────────
# Data & constants
# ──────────────────────────────────────────────────────────────────────────────
DATA_DIR = Path("./data")

MOVIES = pd.read_csv(DATA_DIR / "movie_after_cleaning.csv")
MOVIES_SPLITS = pd.read_excel(DATA_DIR / "splits_movie.xlsx", sheet_name=None)
SERIES = pd.read_csv(DATA_DIR / "series_after_cleaning.csv")
SERIES_SPLITS = pd.read_excel(DATA_DIR / "splits_series.xlsx", sheet_name=None)

DATA_BY_TAB = {
    "movie": (MOVIES, MOVIES_SPLITS),
    "series": (SERIES, SERIES_SPLITS),
}

VISUALIZATION_BUILDERS = {
    "overview": (dash1.generate_visualizations, 4),
    "content_creators": (dash2.generate_visualizations, 4),
    "parental": (dash3.generate_visualizations, 2),
    "year": (dash4.generate_visualizations, 2),
}

# Top-level stats
NUM_WORKS, NUM_COUNTRIES, NUM_LANGUAGES, AVG_VOTES = get_constants(
    MOVIES, SERIES, MOVIES_SPLITS, SERIES_SPLITS
)

MAX_OPTIONS_DISPLAY = 3_300
DROPDOWN_OPTIONS = {
    "movie": [{"label": t, "value": t} for t in MOVIES["title"][:MAX_OPTIONS_DISPLAY]],
    "series": [{"label": t, "value": t} for t in SERIES["title"][:MAX_OPTIONS_DISPLAY]],
}

BRAND_COLOR = "#deb522"

CARD_STYLE = {
    "paddingBlock": "10px",
    "backgroundColor": BRAND_COLOR,
    "border": "none",
    "borderRadius": "10px",
}

TAB_STYLE_IDLE = {
    "borderRadius": "10px",
    "padding": 0,
    "marginInline": "5px",
    "display": "flex",
    "alignItems": "center",
    "justifyContent": "center",
    "fontWeight": "bold",
    "backgroundColor": BRAND_COLOR,
    "border": "none",
}
TAB_STYLE_ACTIVE = {**TAB_STYLE_IDLE, "textDecoration": "underline"}

# ──────────────────────────────────────────────────────────────────────────────
# Helpers
# ──────────────────────────────────────────────────────────────────────────────
def stats_card(title: str, value, img: str) -> html.Div:
    """Single KPI card."""
    return html.Div(
        dbc.Card(
            [
                dbc.CardImg(src=img, top=True, style={"width": "50px", "alignSelf": "center"}),
                dbc.CardBody(
                    [
                        html.P(value, style={"margin": 0, "fontSize": "22px", "fontWeight": "bold"}),
                        html.H4(title, style={"margin": 0, "fontSize": "18px", "fontWeight": "bold"}),
                    ],
                    style={"textAlign": "center"},
                ),
            ],
            style=CARD_STYLE,
        )
    )


def wrap_figures(figures) -> html.Div:
    """Lay out a list of Plotly figures in a 2-column grid."""
    return html.Div(
        [
            html.Div(dcc.Graph(figure=fig), style={"width": "50%", "display": "inline-block"})
            for fig in figures
        ]
    )


In [ ]:
# Initialize the Dash app
app = Dash(
    __name__,  # standard Python name reference
    external_stylesheets=[dbc.themes.BOOTSTRAP],  # loads Bootstrap styles into Dash
    title="IMDB Data Analysis Dashboard"  # sets the browser tab title
)